In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from models.CoAtNet.CoAtNet import CoAtNet as Model
import os
import numpy as np
import pandas
from Frame.Framework import Framework
from Frame.ModelParalellBase import ModelParalellBase
from Data_function.Dataset_folder import Dataset
from Data_function.Transfomer import  Transform
import configparser
from torch.utils.tensorboard import SummaryWriter

In [2]:
# x=torch.randn(1,3,224,224)
# coatnet=Model(num_classes=2)
# y=coatnet(x)
# print(y.shape)


In [3]:
#get data from config file
config = configparser.ConfigParser()
config.read('./config/ConfigFile.properties')
width = int(config["config"]['width'])
height = int(config['config']['height'])
resize = (width, height)
mean = [float(x) for x in config['config']['mean'].split(', ')]
std = [float(x) for x in config['config']['std'].split(', ')]
batch = int(config['config']['batch'])
num_worker = int(config['config']['num_worker'])
lr = float(config['config']['lr'])
epochs = int(config['config']['epochs'])

In [4]:
#config function
framework = Framework()
transformers = Transform(resize=resize, mean= mean, std=std)

opt_func = torch.optim.SGD

Create frame work


In [5]:
#Create dataset
train_dataset = Dataset(root_folder='/home/saplap/Desktop/Model/cifar-100/train', transformer=transformers, phase='train')
val_dataset = Dataset(root_folder='/home/saplap/Desktop/Model/cifar-100/val', transformer=transformers, phase = 'val')
test_dataset = Dataset(root_folder='/home/saplap/Desktop/Model/cifar-100/test', transformer=transformers, phase = 'val')






In [6]:
train_dl = DataLoader(dataset = train_dataset, batch_size = batch, num_workers=num_worker, shuffle=True)
val_dl = DataLoader(dataset = val_dataset, batch_size = batch*2, num_workers=num_worker, shuffle=True)
# test_dl = DataLoader(dataset = test_dataset, batch_size = batch*2, num_workers=num_worker, shuffle=True)

In [7]:
#Create model
model = Model(num_classes= 100)
#Run multiple gpu
# model = ModelParalellBase(model)

/home/saplap/anaconda3/envs/giangnh/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)
loss_function = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
# writer = SummaryWriter("resnet50")

In [9]:
framework.fit(model, train_dl= train_dl, val_dl = val_dl, lr= lr, epochs=epochs, optimizer= optimizer, scheduler = scheduler, loss_func=loss_function, save_each_model='Each_epoch', save_model='output.pt')